In [1]:
import datetime 
from datetime import timedelta 
import os
import glob
import numpy as np
import pandas as pd
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import roc_curve
from torch import nn, optim

import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from itertools import cycle
from tqdm.notebook import tqdm
import re
from sklearn.metrics import roc_auc_score, plot_roc_curve

import matplotlib.pyplot as plt
import seaborn
from collections import defaultdict

from sklearn import svm

In [3]:
# df_israel_covid 이스라엘 데이터셋
# corona_isolation_per_day_ver_0059 일일 코로나 격리자 수 
# time_series_covid19_confirmed_global 누적 확진자 수 

base_path = './opendataset'

df_israel_covid = pd.read_csv(f'{base_path}/israel_covid_large2.csv')

df_small = df_israel_covid.sample(n=df_israel_covid.shape[0] // 100, random_state=1212)
df_small.shape

/Users/mckim/anaconda3/envs/flu_pred/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(69656, 10)

In [4]:

df_small = df_small.sort_values('test_date')
df_small.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
6965613,2020-03-11,1,1,0,0,0,שלילי,NaN,נקבה,Other
6965311,2020-03-11,0,0,0,1,0,שלילי,NaN,זכר,Abroad
6965511,2020-03-11,1,0,1,0,0,שלילי,NaN,זכר,Other
6965355,2020-03-11,1,0,0,1,0,שלילי,NaN,נקבה,Abroad
6965296,2020-03-11,0,0,0,0,0,שלילי,NaN,נקבה,Other


In [5]:
df_small.to_csv(f'{base_path}/israel_covid_small.csv')